# Packed Json Graph with Relations - Steps
A Packed Json Graph is a Json structure that has minimal hierarchy of data and has no graph specific information such as separate edges list

In [16]:
from neo4j import GraphDatabase
import cypher_utils as ctl
from cypher_utils import cypher
import utils as utl

ctl.open("bolt://localhost:7687", "neo4j", "test")

In [25]:
packed_json = utl.load_json("../data/zigbee_packed.json")
len(packed_json)
relations_map = utl.load_json("../data/zigbee_optional_relations.json")
print(relations_map)

{'Coordinators': {'Devices': 'coordinates'}, 'Rooms': {'Devices': 'contains'}}


In [26]:
cypher("match (n) detach delete n")

In [27]:
def stringify_props(name,node,packed_json):
    props = f"name:'{name}'"
    for key,prop_val in node.items():
        if(key in packed_json):#This is a relationship not a property, so skip in node creation
            continue
        if(type(node[key]) == str):
            props += f", {key}:'{node[key]}'"
        else:
            props += f", {key}:{node[key]}"
    return props

def stringify_relations_map(source,Label1,node,packed_json,relations_map):
    relations_cypher = ""
    for Label2,prop_val in node.items():
        if(Label2 in packed_json):
            assert(isinstance(prop_val,list))
            for target in prop_val:
                rel = "relatesto"
                #print(f"{Label1} => {Label2}")
                if(Label1 in relations_map) and (Label2 in relations_map[Label1]):
                    rel = relations_map[Label1][Label2]
                relations_cypher += f"\n (`{source}`)-[:{rel}]->(`{target}`),"
    return relations_cypher

In [28]:
def json_packed_to_cypher(packed_json,relations_map=None):
    cmd = "CREATE "
    #Nodes
    for Label,Labels_map in packed_json.items():
        for name,node in Labels_map.items():
            props = stringify_props(name,node,packed_json)
            cmd += f"\n (`{name}`:{Label} {{{props}}}),"
    #Relationships
    for Label,Labels_map in packed_json.items():
        for name,node in Labels_map.items():
            if(relations_map is not None):
                cmd += stringify_relations_map(name,Label,node,packed_json,relations_map)
            else:
                cmd += stringify_relations(name,node,packed_json)
    cmd = cmd[:-1]+";"
    return cmd

In [29]:
cmd = json_packed_to_cypher(packed_json,relations_map)
print(cmd)

CREATE 
 (`lzig`:Coordinators {name:'lzig', hostname:'lifo', zigbee_channel:13}),
 (`mzig`:Coordinators {name:'mzig', hostname:'mano', zigbee_channel:26}),
 (`bedroom`:Rooms {name:'bedroom'}),
 (`balcony`:Rooms {name:'balcony'}),
 (`livingroom`:Rooms {name:'livingroom'}),
 (`bathroom`:Rooms {name:'bathroom'}),
 (`office`:Rooms {name:'office'}),
 (`hallway`:Rooms {name:'hallway'}),
 (`kitchen`:Rooms {name:'kitchen'}),
 (`mano power`:Devices {name:'mano power', mac:'0x00124b001d4a012d'}),
 (`floody`:Devices {name:'floody', mac:'0x00158d000255fb8b'}),
 (`desk motion`:Devices {name:'desk motion', mac:'0x00158d0002a4f61e'}),
 (`cube`:Devices {name:'cube', mac:'0x00158d00028f8531'}),
 (`bed light button`:Devices {name:'bed light button', mac:'0x00158d0001ef6374'}),
 (`bed weather`:Devices {name:'bed weather', mac:'0x00158d00026ecd6a'}),
 (`balcony weather`:Devices {name:'balcony weather', mac:'0x00158d000237df83'}),
 (`balcony window left`:Devices {name:'balcony window left', mac:'0x00158d00

In [30]:
utl.save_file("zigbee_relations.cyp",cmd)

5791 bytes saved in zigbee_relations.cyp


In [31]:
cypher(cmd)